In [1]:
import nbimporter
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tqdm import tqdm
import copy
import time
import utils
import warnings
import rits_i
import brits_i
import rits
import brits
import argparse
import easydict
import data_loader
import pandas as pd
import ujson as json

from sklearn import metrics

from ipdb import set_trace

# easydict version
args = easydict.EasyDict({
    "epochs": 200,
    "batch_size": 32,
    "model":"model"
})

def train_test(model):
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)
    data_iter = data_loader.get_loader(file = "E:/Python/tr_seoul.json", batch_size = args.batch_size)
    output=False
    
    prog = tqdm(range(args.epochs))
    result=[]
    
    for i in prog:
        model.train()
        run_loss = 0.0
        
        for idx, data in enumerate(data_iter):
            data = utils.to_var(data)
            ret = model.run_on_batch(data, optimizer,48)
            run_loss += ret['loss'].data
            
            prog.set_description("loss: {:0.3f}".format(run_loss))

        result=evaluate(model, data_iter)
#         prog.set_description("loss: {:0.3f}, MAE: {:0.3f}, RMSE: {:0.3f}".format(run_loss,result['MAE'],result["RMSE"]))
        print("loss: {:0.3f}, MAE: {:0.3f}, RMSE: {:0.3f}".format(run_loss,result['MAE'],result["RMSE"])) # temp
    
#     print(result)
    
    data_iter = data_loader.get_loader(file = "E:/Python/te_seoul.json", batch_size = args.batch_size)
    output=True
    test_result=evaluate(model, data_iter,predict=output, testmode=True)
    
    return(test_result)

def evaluate(model, val_iter, predict=False, testmode=False):
#     model.eval()

    evals = []
    imputations = []
    arr_imp = np.array(1)
    
    with torch.no_grad():
        model.eval()
        for idx, data in enumerate(val_iter):
            data = utils.to_var(data)
            if testmode==True : ret = model.run_on_batch(data, None, 24)
            else : ret = model.run_on_batch(data, None, 48)

            eval_masks = ret['eval_masks'].data.cpu().numpy()
            eval_ = ret['evals'].data.cpu().numpy()
            imputation = ret['imputations'].data.cpu().numpy()
            evals += eval_[np.where(eval_masks == 1)].tolist()
            imputations += imputation[np.where(eval_masks == 1)].tolist()

            if idx==0: arr_imp=copy.deepcopy(imputation.squeeze())
            else: arr_imp = np.concatenate((arr_imp,imputation.squeeze()),axis=0)

        evals = np.asarray(evals)
        imputations = np.asarray(imputations)
        r2 = r2_score(evals,imputations)
        cor = np.corrcoef(evals,imputations)[0, 1]

    result={"MAE":np.abs(evals - imputations).mean(),"RMSE":np.sqrt(np.mean((evals - imputations)**2)), "R2":r2, "Cor":cor}
    
    if testmode: 
        print("\nTest MAE: {:0.3f}, Test RMSE: {:0.3f}, R-squared: {:0.3f}, Cor: {:0.3f}".format(result['MAE'],result["RMSE"],result["R2"],result['Cor']))
#         if predict: predict_result(arr_imp)
    
    return(result)

def predict_result(impute_map, tp):
    imputation = np.array(evaluate(model, val_iter,1))
    df = pd.read_csv("E:/데이터모음/대기오염/preprocess/seoul_pm25.csv")

    print(df.shape)
    print(imputation.shape)
    
    if tp==0 : md="rits_i" 
    elif tp==1 : md = "brits_i"
    elif tp==2 : md = "rits"
    else : md = "brits"
        
    colnm = ["111121","111123","111131","111141","111142","111151","111152","111161","111171","111181","111191",
             "111201","111212","111221","111231","111241","111251","111261","111262","111273","111274","111281",
             "111291","111301","111311"]
    pd.DataFrame(dff,columns=colnm, index=df.iloc[:,1]).to_csv('E:/데이터모음/대기오염/preprocess/new_impute_{}.csv'.format(md), sep=',', na_rep='NaN')
    print("predict_out")
    return 0

def parse_delta(masks, dir_):
    if dir_ == 'backward':
        masks = masks[::-1]

    deltas = []

    for h in range(masks.shape[1]):
        if h == 0:
            deltas.append(1)
        else:
            deltas.append(1 + (1 - masks[0,h]) * deltas[-1])

    return np.array(deltas)

def parse_rec(values, masks, evals, eval_masks, dir_):
    deltas = parse_delta(masks, dir_)

    # only used in GRU-D
#     forwards = pd.DataFrame(values).fillna(method='ffill').fillna(0.0).values

    values = np.array(values,dtype=float)
    values[np.isnan(values)] = 0
    evals = np.array(evals,dtype=float)
    evals[np.isnan(evals)] = 0
    
    rec = {}
    rec['values'] = values.tolist()[0]
    rec['masks'] = masks.astype('int32').tolist()[0]
    rec['evals'] = evals.tolist()[0]
    rec['eval_masks'] = eval_masks.astype('int32').tolist()[0]
    rec['deltas'] = deltas.tolist()
    
    return rec

def makedata(tr_scale_df_path, tr_scale_df_miss_path,te_scale_df_path, te_scale_df_miss_path):
    tr_scale_df = pd.read_csv(tr_scale_df_path)
    tr_scale_df_miss = pd.read_csv(tr_scale_df_miss_path)
    te_scale_df = pd.read_csv(te_scale_df_path)
    te_scale_df_miss = pd.read_csv(te_scale_df_miss_path)
    tr_length = int(tr_scale_df.shape[0])
    te_length = int(te_scale_df.shape[0])
    
    ######### train json file composition
    # seq_len = 48, rows = 548
    # evals = tr_scale_df, values = tr_scale_df_miss

    fs = open('E:/Python/tr_seoul.json', 'w')
    
    tr_nrow = 548
    for i in range(tr_nrow):
        rec={}
        f_list, b_list = [[] for i in range(2)]
        for j in range(int(tr_length/tr_nrow)):
#             evals,values=[[] for i in range(2)]
            evals=[]; values=[]
            evals.append(tr_scale_df.iloc[j+int(tr_length/tr_nrow)*i,range(2,27)])
            values.append(tr_scale_df_miss.iloc[j+int(tr_length/tr_nrow)*i,range(2,27)])

            shp = np.array(evals).shape # do we need this variable?
            masks = ~np.isnan(np.array(values,dtype=np.float))
            eval_masks = (~np.isnan(np.array(values,dtype=np.float))) ^ (~np.isnan(np.array(evals,dtype=np.float)))
            
            masks = masks.reshape(shp)
            eval_masks = eval_masks.reshape(shp)
            
            # prepare the model for both directions
            f_list.append(parse_rec(values, masks, evals, eval_masks, dir_='forward'))
            b_list.append(parse_rec(values[::-1], masks[::-1], evals[::-1], eval_masks[::-1], dir_='backward'))

        rec["forward"] = f_list; rec["backward"] = b_list
        rec = json.dumps(rec)
        
        fs.write(rec)
        fs.write("\n")
        
    print("tr_seoul")
    fs.close()
    
    ######### test json file composition
    # seq_len = 24, rows = 365
    # evals = te_scale_df, values = te_scale_df_miss
    
    fs = open('E:/Python/te_seoul.json', 'w')
    
    te_nrow = 365
    for i in range(te_nrow):
        rec={}
        f_list, b_list = [[] for i in range(2)]
        for j in range(int(te_length/te_nrow)):
            evals=[]; values=[]
            evals.append(te_scale_df.iloc[j+int(te_length/te_nrow)*i,range(2,27)])
            values.append(te_scale_df_miss.iloc[j+int(te_length/te_nrow)*i,range(2,27)])

            shp = np.array(evals).shape # do we need this variable?
            masks = ~np.isnan(np.array(values,dtype=np.float))
            eval_masks = (~np.isnan(np.array(values,dtype=np.float))) ^ (~np.isnan(np.array(evals,dtype=np.float)))
            
            masks = masks.reshape(shp)
            eval_masks = eval_masks.reshape(shp)
            
            # prepare the model for both directions
            f_list.append(parse_rec(values, masks, evals, eval_masks, dir_='forward'))
            b_list.append(parse_rec(values[::-1], masks[::-1], evals[::-1], eval_masks[::-1], dir_='backward'))

        rec["forward"] = f_list; rec["backward"] = b_list
        rec = json.dumps(rec)
        
        fs.write(rec)
        fs.write("\n")
        
    print("te_seoul")
    fs.close()
    
    return tr_length

def run():
    warnings.filterwarnings(action='ignore')

    # 1) PM2.5
    makedata('E:/데이터모음/대기오염/preprocess/tr_seoul_pm25.csv',
             'E:/데이터모음/대기오염/preprocess/tr_seoul_pm25_miss.csv',
             'E:/데이터모음/대기오염/preprocess/te_seoul_pm25.csv',
             'E:/데이터모음/대기오염/preprocess/te_seoul_pm25_miss.csv')
    # 2) PM10
#             makedata('E:/데이터모음/대기오염/preprocess/scale_seoul_pm10.csv','E:/데이터모음/대기오염/preprocess/scale_seoul_pm10_miss.csv')        
    for j in range(2):
        if j==0 : 
            model = rits.Model()
            print("\nrits")
        else : 
            model = brits.Model()
            print("\nbrits")

        if torch.cuda.is_available():
            model = model.cuda()
        
        train_test(model)
        

if __name__ == '__main__':
    run()


  0%|                                                                                          | 0/200 [00:00<?, ?it/s]


rits


loss: 816.455:   0%|▎                                                                | 1/200 [00:45<2:29:55, 45.20s/it]

loss: 816.455, MAE: 10.282, RMSE: 15.287


loss: 593.680:   1%|▋                                                                | 2/200 [01:29<2:27:16, 44.63s/it]

...

loss: 668.943: 100%|██████████████████████████████████████████████████████████████▋| 199/200 [2:25:38<00:43, 43.84s/it]

loss: 668.943, MAE: 7.081, RMSE: 10.440


loss: 663.203: 100%|███████████████████████████████████████████████████████████████| 200/200 [2:26:22<00:00, 43.91s/it]

loss: 663.203, MAE: 7.120, RMSE: 10.498



Test MAE: 5.617, Test RMSE: 8.467, R-squared: 0.801, Cor: 0.896
